In [ ]:
import os
import sys
import pickle
from pathlib import Path

import fdsvismap as fv
import jupedsim as jps
import matplotlib.pyplot as plt
import pedpy
import shapely
from matplotlib.patches import Circle
from shapely import Polygon, from_wkt
import numpy as np
from jupedsim.internal.notebook_utils import animate, read_sqlite_file
from typing import Any, List, Tuple
import math

## TODO

- Switch to the journey that has more visible wps on route.


## Config parameters


In [ ]:
num_agents = 21
premovement_time = 300  # seconds
v0 = 1.0
seed = 1
c0 = 3.0
# Set times when the simulation should be evaluated
update_time = 1
max_vis_simulation_time = 1000  # FDS simulation time in seconds
times = range(premovement_time, max_vis_simulation_time, update_time)
trajectory_file = f"output_N{num_agents}.sqlite"  # output file


distance_to_waypoints = 0.5
# waypoints: (x, y, angle)
waypoints = [
    (13.5, 8.5, 0),  # 0
    (10.5, 4.5, 180),  # 1
    (18.5, 6.5, 270),  # 2
    (25, 14.5, 180),  # 3
    (4, 6.5, 90),  # 4
    (2.5, 14.5, 180),  # 5
]

## Vismap config


In [ ]:
def load_or_compute_vis(sim_dir, waypoints, times, pickle_path):
    if pickle_path.is_file():
        try:
            with open(pickle_path, "rb") as file:
                vis = pickle.load(file)
            print("Vis object loaded successfully.")
        except FileNotFoundError:
            sys.exit(f"No file found at {pickle_path}, please check the file path.")

        except Exception as e:
            sys.exit(f"An error occurred while loading the visualization: {e}")
    else:
        # Read and process data if not existing
        vis = fv.VisMap()
        vis.read_fds_data(str(sim_dir))
        vis.set_start_point(8, 8)

        for wp in waypoints:
            vis.set_waypoint(x=wp[0], y=wp[1], c=c0, alpha=wp[2])

        # Set waypoint for exit
        # vis.set_waypoint(x=exits[1].centroid.xy[0][0], y=exits[1].centroid.xy[1][0], c=c0, alpha=180)

        # Define time points and compute the visualization
        vis.set_time_points(times)
        vis.compute_all()

        # Save results to pickle file
        pickle_path.parent.mkdir(exist_ok=True)  # Ensure directory exists
        try:
            # Serialize the visualization object to a pickle file
            with open(pickle_path, "wb") as file:
                pickle.dump(vis, file)
                print("Visualization saved successfully.")
        except Exception as e:
            sys.exit(f"An error occurred while saving the visualization: {e}")

    return vis


project_root = Path(os.path.abspath("")).parent
sim_dir = project_root / "fds_data"
pickle_path = project_root / "processed_data" / "vismap.pkl"
vis = load_or_compute_vis(sim_dir, waypoints, times, pickle_path)

In [ ]:
fig, ax = vis.create_aset_map_plot(plot_obstructions=True)

In [ ]:
fig, ax = vis.create_time_agg_wp_agg_vismap_plot(plot_obstructions=True)

# Check if waypoint is visible from given location at given time

x = 18.51
y = 6.79
t = 16
for wp_id, _ in enumerate(waypoints):
    # for t in [0, 13, 100, 600, 850]:
    # wp_id = 2
    print(f" {t = },  {wp_id = }")
    print(vis.wp_is_visible(time=t, x=x, y=y, waypoint_id=wp_id))
    # Get distance from waypoint to given location
    print(f"Distance: {vis.get_distance_to_wp(x=x, y=y, waypoint_id=wp_id):.2f} [m]")
    # Calculate local visibility at given location and time, considering a specific c factor
    # print("local visibility:", vis.get_local_visibility(time=100, x=x, y=y, c=3))
    print("----")

In [ ]:
v = []
c = 3
for t in times:
    v.append(vis.get_local_visibility(time=t, x=25, y=6, c=c))
    # v.append(vis.get_local_visibility(time=t, x=7, y=4, c=3))

plt.plot(times, v)
plt.xlabel("Time [s]")
plt.ylabel("Local Visibility")

In [ ]:
def calculate_desired_speed(v, c, max_speed, range=2.0):
    # Calculate the proximity of v to c, and adjust speed accordingly
    if v <= c:
        return 0
    else:
        # Create a linear relation or any other functional form as needed
        # This example uses an exponential decay to decrease speed as v approaches c
        return max_speed * (1 - np.exp(-(v - c) / range))


desired_speeds = [
    calculate_desired_speed(visibility, c + 2, max_speed=1.0, range=5)
    for visibility in v
]
plt.plot(times, desired_speeds)
plt.xlabel("Time [s]")
plt.ylabel("Desired Speed [m/s]")

In [ ]:
# CO2 = np.arange(0, 10, 0.001)
# HV = 0.141 * np.exp(0.1930* CO2+ 2.0004)
# plt.plot(CO2, HV)

In [ ]:
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)


def get_next_waypoint(point, waypoints):
    """Return the waypoint that is closest to point."""
    min_distance = float("inf")
    next_waypoint = None
    next_waypoint_id = -1
    # print(point)
    for wp in waypoints:
        d = distance(point, wp)
        if d < min_distance:
            min_distance = d
            next_waypoint = wp
            next_waypoint_id = waypoints.index(wp)

    # print(f"Next waypoint: {next_waypoint}")
    return next_waypoint_id, next_waypoint

In [ ]:
def plot_simulation_configuration(
    walkable_area, starting_positions, exits, path1=None, path2=None
):
    axes = pedpy.plot_walkable_area(walkable_area=walkable_area)
    for exit_area in exits:
        axes.fill(*exit_area.exterior.xy, color="indianred")

    for starting_position in starting_positions:
        axes.scatter(*zip(*starting_position), s=1, color="gray")

    axes.set_xlabel("x/m")
    axes.set_ylabel("y/m")
    axes.set_aspect("equal")
    for idx, waypoint in enumerate(waypoints):
        axes.plot(waypoint[0], waypoint[1], "go")
        axes.annotate(
            f"$WP_{idx}$",
            (waypoint[0], waypoint[1]),
            textcoords="offset points",
            xytext=(10, -15),
            ha="center",
        )
        circle = Circle(
            (waypoint[0], waypoint[1]),
            distance_to_waypoints,
            fc="green",
            ec="green",
            alpha=0.1,
        )
        axes.add_patch(circle)

    if path1:
        path1 = path1
        x_coords = [point[0] for point in path1]
        y_coords = [point[1] for point in path1]
        for idx, point in enumerate(path1):
            if idx == 0:
                color="black"
                facecolor="black"
            #else:
            #    color = "blue"
            #    facecolor="none"    
            
                axes.plot(
                    point[0],
                    point[1],
                    "o",
                    markerfacecolor=facecolor,
                    markeredgewidth=1,
                    markeredgecolor=color,
                )
        axes.plot(x_coords, y_coords, "b--")

    if path2:
        path2 = path2[:]
        x_coords = [point[0] for point in path2]
        y_coords = [point[1] for point in path2]
        for idx, point in enumerate(path2):
            if idx == 0:
                color="black"
                facecolor="black"
                axes.plot(
                    point[0],
                    point[1],
                    "o",
                    markerfacecolor=facecolor,
                    markeredgewidth=1,
                    markeredgecolor=color,
                )
        axes.plot(x_coords, y_coords, "r--")

In [ ]:
with open("geometry.wkt", "r") as file:
    data = file.readlines()

wkt_data = from_wkt(data)
area = wkt_data[0]
obstacles = wkt_data[1:]
obstacle = shapely.union_all(obstacles)
walkable_area = pedpy.WalkableArea(shapely.difference(area, obstacle))
routing = jps.RoutingEngine(walkable_area.polygon)
spawning_area1 = Polygon([(1, 0), (1, 3), (15, 3), (15, 0)])
spawning_area2 = Polygon([(5, 10), (19, 10), (19, 14.5), (5, 14.5)])

exits = [
    # left
    Polygon([(1, 15), (4, 15), (4, 16.0), (1, 16.0), (1, 15)]),
    # right
    Polygon([(24, 15.0), (27, 15.0), (27, 16.0), (24, 16.0), (24, 15.0)]),
]
primary_exit = (exits[1].centroid.xy[0][0], exits[1].centroid.xy[1][0])
secondary_exit = (exits[0].centroid.xy[0][0], exits[0].centroid.xy[1][0])
pos_in_spawning_areas = [
    jps.distributions.distribute_by_number(
        polygon=spawning_area2,
        number_of_agents=num_agents,
        distance_to_agents=0.4,
        distance_to_polygon=0.3,
        seed=seed,
    ),
    jps.distributions.distribute_by_number(
        polygon=spawning_area1,
        number_of_agents=num_agents,
        distance_to_agents=0.4,
        distance_to_polygon=0.3,
        seed=seed,
    ),
]
#starting_point = pos_in_spawning_areas[0][12]
starting_point = waypoints[0][0:2]
starting_point = (10.6, 6.89)
#starting_point = (18, 7)
#starting_point = (10, 12)
starting_point = (2, 7.5)
starting_point = ( 18.51,6.79)
path1 = routing.compute_waypoints(starting_point, primary_exit)
path2 = routing.compute_waypoints(starting_point, secondary_exit)
print("path1:", path1[1:-1])
print("path2:", path2[1:-1])
plot_simulation_configuration(
    walkable_area, pos_in_spawning_areas, exits, path1, path2
)
fig, ax = plt.subplots()
vis1 = []
vis2 = []
times = range(0, 2000,1000)
for time in times:
    print("time", time)
    waypoints_info = compute_waypoints_and_visibility(
                starting_point, primary_exit, waypoints, time
            )
    #print(waypoints_info[0:])

    vis_point = 0
    seen = set()    
    for wp_id, is_visible in zip(waypoints_info[1], waypoints_info[2]):
        if wp_id not in seen and is_visible:
            seen.add(wp_id)
            print(f"{wp_id = }, {is_visible = }")
            x = waypoints[wp_id][0]
            y = waypoints[wp_id][1]
            vis_point += vis.get_local_visibility(time=time, x=x, y=y, c=3)
        
    print("==>> local visibility:", vis_point)
    print("------ SECOND PATH -------")
    vis1.append(vis_point)
    waypoints_info2 = compute_waypoints_and_visibility(
                starting_point, secondary_exit, waypoints, time
            )
    print(waypoints_info2)
    vis_point2 = 0
    seen2 = set()    
    for wp_id, is_visible in zip(waypoints_info2[1], waypoints_info2[2]):
        if wp_id not in seen2 and is_visible:            
            print(f"{wp_id = }, {is_visible = }")
            seen2.add(wp_id)
            x = waypoints[wp_id][0]
            y = waypoints[wp_id][1]
            vis_point2 += vis.get_local_visibility(time=time, x=x, y=y, c=3)

    print("local visibility:", vis_point2)
    vis2.append(vis_point2)


ax.plot(times, vis1, "-",label="Primary")
ax.plot(times, vis2, "-",label="Secondary")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Local Visibility Along Path")  
ax.grid(alpha=0.3) 
plt.legend()

In [ ]:
def log_path_info(time, path, agent, waypoints_info, speed):
    print("path: ", path[1:-1])
    wp_ids = waypoints_info[1]
    wp_visibility = waypoints_info[2]
    wp = waypoints_info[0]
    for wp_id, wp, wp_is_visible in zip(wp_ids, wp, wp_visibility):
        print(
            f"time = {time:.2f}, x = {agent.position[0]:.2f}, y = {agent.position[1]:.2f}, "
            f"v = {speed:.2f}, agent.id = {agent.id}, wp_id = {wp_id}, wp = {wp}, wp_is_visible = {wp_is_visible}, "
            f"agent.model.desiredSpeed = {agent.model.desiredSpeed:.2f}",
            end="\n",
        )
    print("RESULT: ", any(waypoints_info[2]))
    print("===========")
    
def process_waypoints(waypoints_info: List, waypoints: List, vis: Any, time: float)->Tuple[float, float]:
    vis_point = 0
    seen = set()
    for wp_id, is_visible in zip(waypoints_info[1], waypoints_info[2]):
        if wp_id not in seen and is_visible:
            seen.add(wp_id)
            x = waypoints[wp_id][0]
            y = waypoints[wp_id][1] 
            vis_point += vis.get_local_visibility(time=time, x=x, y=y, c=3)
    return vis_point

def compute_waypoints_and_visibility(agent_position: Tuple[float, float], primary_exit: Tuple[float, float], waypoints: List[Tuple[float, float, float]], time: float):
    path = routing.compute_waypoints(agent_position, primary_exit)
    wps_on_path = []
    wps_on_path_id = []
    wps_on_path_visibility = []
    # if len(path[1:-1]) == 1:  # this is then right before the exit
    #     # return the nearest wp
    #     for wp_id, wp in enumerate(waypoints):
    #     return waypoints[3], [3], [True]
     
    for point in path[1:]:
        wp_id, wp = get_next_waypoint(point=point, waypoints=waypoints)
        wps_on_path.append(wp)
        wps_on_path_id.append(wp_id)
        distance = vis.get_distance_to_wp(
            x=agent_position[0], y=agent_position[1], waypoint_id=wp_id
        )
        if True or distance > 0.2:
            wp_is_visible = vis.wp_is_visible(
                time=time,
                x=agent_position[0],
                y=agent_position[1],
                waypoint_id=wp_id,
            )
        else:
            wp_is_visible = True
        wps_on_path_visibility.append(wp_is_visible)

    return wps_on_path, wps_on_path_id, wps_on_path_visibility
def check_and_update_journeys(
    simulation: jps.Simulation,
    time: float,
    primary_exit: Tuple[float, float],
    secondary_exit: Tuple[float, float],
    primary_journey_id: int,
    secondary_journey_id: int,
    primary_exit_id: int,
    secondary_exit_id: int,
    waypoints: List[Tuple[float, float, float]],
    vis: Any,
):

    for agent in simulation.agents():
        agent_position = agent.position
        waypoints_info = compute_waypoints_and_visibility(
            agent_position, primary_exit, waypoints, time
        )
        waypoints_info_s = compute_waypoints_and_visibility(
            agent_position, secondary_exit, waypoints, time
        )
        
        # Set the speed
        local_visibility = vis.get_local_visibility(
            time=time, x=agent_position[0], y=agent_position[1], c=c0
        )
        agent.model.desiredSpeed = calculate_desired_speed(
            local_visibility, c0, v0, range=5
        )
        vis_point = process_waypoints(waypoints_info, waypoints, vis, time)
        vis_point_s = process_waypoints(waypoints_info_s, waypoints, vis, time)
        if False and agent_position[0] > 18:
            print("Path RIGHT")
            print(waypoints_info[1:])
            log_path_info(
                time,
                routing.compute_waypoints(agent_position, primary_exit),
                agent,
                waypoints_info,
                agent.model.desiredSpeed,
            )
            print("Path LEFT")
            print(waypoints_info_s[1:])
            log_path_info(
                time,
                routing.compute_waypoints(agent_position, secondary_exit),
                agent,
                waypoints_info_s,
                agent.model.desiredSpeed,
            )
            print(f"{vis_point = }, {vis_point_s = }")
        if vis_point_s > vis_point:
            simulation.switch_agent_journey(
                agent.id, secondary_journey_id, secondary_exit_id
            )
        else:
            simulation.switch_agent_journey(
                agent.id, primary_journey_id, primary_exit_id
            )    

        # Check if no waypoints on path are visible, then switch the journey
        # if not any(waypoints_info[2]):
        #     if agent_position[0] > 19:
        #         print(waypoints_info[2])
        #         log_path_info(
        #             time,
        #             routing.compute_waypoints(agent_position, primary_exit),
        #             agent,
        #             waypoints_info,
        #             agent.model.desiredSpeed,
        #         )
        #         print(f"=======>> Agent {agent.id} is not visible at time {time}")
        #     simulation.switch_agent_journey(
        #         agent.id, secondary_journey_id, secondary_exit_id
        #     )
        # else:
        #     simulation.switch_agent_journey(
        #         agent.id, primary_journey_id, primary_exit_id
        #     )

In [ ]:
def add_agents_to_simulation(
    simulation: jps.Simulation, pos_in_spawning_areas, journey_primary_id, exit_ids
):
    # Add agents to the upper room
    ids_up = set(
        [
            simulation.add_agent(
                jps.SocialForceModelAgentParameters(
                    desiredSpeed=0,
                    radius=0.1,
                    journey_id=journey_primary_id,
                    stage_id=exit_ids[1],
                    position=pos,
                )
            )
            for pos in pos_in_spawning_areas[0]
        ]
    )
    # Add agents to the lower room
    ids_down = set(
        [
            simulation.add_agent(
                jps.SocialForceModelAgentParameters(
                    desiredSpeed=0,
                    radius=0.1,
                    journey_id=journey_primary_id,
                    stage_id=exit_ids[1],
                    position=pos,
                )
            )
            for pos in pos_in_spawning_areas[1]
        ]
    )
    return ids_up, ids_down

In [ ]:
def run_simulation(trajectory_file, walkable_area, exits, vis):
    simulation = jps.Simulation(
        model=jps.SocialForceModel(),
        geometry=walkable_area.polygon,
        trajectory_writer=jps.SqliteTrajectoryWriter(output_file=Path(trajectory_file)),
    )
    exit_ids = [
        simulation.add_exit_stage(exit_area.exterior.coords[:-1]) for exit_area in exits
    ]
    # Notausgang
    journey_secondary = jps.JourneyDescription([exit_ids[0]])
    journey_secondary_id = simulation.add_journey(journey_secondary)
    secondary_exit = (exits[0].centroid.xy[0][0], exits[0].centroid.xy[1][0])
    # Hauptausgang
    journey_primary = jps.JourneyDescription([exit_ids[1]])
    journey_primary_id = simulation.add_journey(journey_primary)
    primary_exit = (exits[1].centroid.xy[0][0], exits[1].centroid.xy[1][0])
    # Add agents to the upper room
    add_agents_to_simulation(
        simulation, pos_in_spawning_areas, journey_primary_id, exit_ids
    )
    # Simulate premovement time in seconds
    premovement_iterations = premovement_time * int(1 / simulation.delta_time())
    simulation.iterate(premovement_iterations)
    # Start movement. The simulation will stop if no agents are left or the max_vis_simulation_time is reached
    while simulation.elapsed_time() <  premovement_time + max_vis_simulation_time:
        t = simulation.elapsed_time()  # seconds
        # Generate new agents every 10 seconds
        if (
            simulation.iteration_count() > premovement_iterations + 1000
            and simulation.iteration_count() % 2000 == 0
        ):
            add_agents_to_simulation(
                simulation, pos_in_spawning_areas, journey_primary_id, exit_ids
            )

        if simulation.iteration_count() % 2000 == 0:
            print(f"Simulation time: {t:2.2f} s", end="\r")
            check_and_update_journeys(
                simulation=simulation,
                time=t,
                primary_exit=primary_exit,
                primary_exit_id=exit_ids[1],
                primary_journey_id=journey_primary_id,
                secondary_exit=secondary_exit,
                secondary_exit_id=exit_ids[0],
                secondary_journey_id=journey_secondary_id,
                waypoints=waypoints,
                vis=vis,
            )

        simulation.iterate()

    print(f"Simulation finished after {simulation.elapsed_time():.2f} seconds.")

In [112]:
premovement_time = 400  # seconds
run_simulation(
    walkable_area=walkable_area, exits=exits, trajectory_file=trajectory_file, vis=vis
)
trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area, every_nth_frame=10)

In [ ]:
animate(trajectory_data, walkable_area, every_nth_frame=1)

# Plot Trajectories


In [ ]:
# from pedpy import plot_walkable_area
# from pedpy.column_identifier import (
#     ID_COL,
#     X_COL,
#     Y_COL,
# )

# def plot_trajectories(traj_data, walkable_area = None, axes = None, **kwargs):
#     traj_color = kwargs.pop("traj_color", 'red')
#     traj_width = kwargs.pop("traj_width", 1.0)
#     traj_alpha = kwargs.pop("traj_alpha", 1.0)

#     traj_start_marker = kwargs.pop("traj_start_marker", "")
#     traj_end_marker = kwargs.pop("traj_end_marker", "")

#     if axes is None:
#         axes = plt.gca()

#     if walkable_area is not None:
#         axes = plot_walkable_area(
#             walkable_area=walkable_area, axes=axes, **kwargs
#         )

#     for _, ped in traj_data.groupby(ID_COL):
#         axes.plot(
#             ped[X_COL],
#             ped[Y_COL],
#             alpha=traj_alpha,
#             color=traj_color,
#             linewidth=traj_width,
#         )
#         axes.scatter(
#             ped[ped.frame == ped.frame.min()][X_COL],
#             ped[ped.frame == ped.frame.min()][Y_COL],
#             color=traj_color,
#             marker=traj_start_marker,
#         )
#         axes.scatter(
#             ped[ped.frame == ped.frame.max()][X_COL],
#             ped[ped.frame == ped.frame.max()][Y_COL],
#             color=traj_color,
#             marker=traj_end_marker,
#         )

#     axes.set_xlabel(r"x/m")
#     axes.set_ylabel(r"y/m")

#     return axes


# trajectory_data_blue = trajectory_data.data[trajectory_data.data['id'].isin(smoke_non_affected_agent_ids)]
# trajectory_data_red = trajectory_data.data[trajectory_data.data['id'].isin(smoke_affected_agent_ids)]

# plot_trajectories(
#     traj_data=trajectory_data_blue,
#     walkable_area= walkable_area,
#     traj_alpha=0.5,
#     traj_width=1,
#     traj_start_marker='.',
#     traj_color='blue'
# ).set_aspect("equal")
# plot_trajectories(
#     traj_data=trajectory_data_red,
#     traj_alpha=0.5,
#     traj_width=1,
#     traj_start_marker='.',
#     traj_color='red'
# ).set_aspect("equal")
# plt.plot([], [], 'r', label='smoke affected agents')
# plt.plot([], [], 'b', label='smoke non affected agents')
# plt.legend()
# plt.xlim(0, 30)
# plt.ylim(0, 15)
# plt.xlabel("X / m")
# plt.ylabel("Y / m")
# plt.yticks([0, 5, 10, 15])
# plt.tight_layout()
# plt.savefig('trajectories.png', dpi=300, bbox_inches='tight')